#### Import Libraries

In [ ]:
import os
import pandas as pd

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dte
from dash.dependencies import Input, Output, State

import plotly.graph_objs as go
from plotly import tools

import numpy as np
from sklearn.preprocessing import LabelEncoder

#### Set Log File

In [ ]:
LOGFILE = 'C:/Users/596596/Documents/GitHub/UVA-DataPalooza/data/predictions_for_plotly.csv'
DATA = pd.read_csv(LOGFILE)

labels = DATA['Actual'].append(DATA['Predictions'])

le = LabelEncoder()
le.fit(labels)

DATA['Actual Encoded'] = le.transform(DATA['Actual'])
DATA['Predictions Encoded'] = le.transform(DATA['Predictions'])

dataframes = {'DATA': DATA}

#### Create Plotly Dash Application & Server

In [ ]:
app = dash.Dash("UVA DataPalooza Plotly Dash Appication", static_folder='assets')
server = app.server

#### Create Data Object Method

In [ ]:
def get_data_object(user_selection):
    """
    For user selections, return the relevant in-memory data frame.
    """
    return dataframes[user_selection]

#### Create Application Layout

In [ ]:
app.layout = html.Div([
    html.Div([
        
        html.Div([
            html.Div([
                html.Img(src='/assets/uva-datapalooza-2018.png', style={'width':'100%', 'height':'100%'})
            ], className='six columns'),
            
            html.Div([
                html.H2('Plotly Dash Demonstration'),
                html.H2('Stack Over Flow Questions Analysis')
            ], className='five columns offset-by-one'),
            
        ], className='row'),
        
        html.Br([]),
        
        dcc.Dropdown(
            id='field-dropdown',
            options=[{'label': df, 'value': df} for df in dataframes],
            value='DATA',
            clearable=False),
        
        html.Br([]),
        
        dte.DataTable(
            rows=[{}],
            row_selectable=False,
            filterable=True,
            sortable=True,
            selected_row_indices=[],
            id='table'),
        
        html.Br([]),
        
        html.Div([
            
            html.Div([
                dcc.Graph(id='simple_graph')                
            ], className='eleven columns')
            
        ]),
        
        html.Br([]),
        
        html.Div([
        
            html.Div([
                dcc.Graph(id='actual_histogram')
            ], className='four columns'),
            
            html.Div([
                dcc.Graph(id='predictions_histogram')
            ], className='four columns'),
            
            html.Div([
                dcc.Graph(id='overlay_histogram')
            ], className='four columns')
            
        ], className='row'),
        
        html.Br([]),
        
        html.Div([
        
            html.Div([
                dcc.Graph(id='density_graph')
            ], className='twelve columns')
            
        ], className='row'),
        
    ], className='eleven columns')

], className='container')

#### Create Application Callbacks

In [ ]:
@app.callback(Output('table', 'rows'), [Input('field-dropdown', 'value')])
def update_table(user_selection):
    df = get_data_object(user_selection)
    return df.to_dict('records')

@app.callback(Output('simple_graph', 'figure'), [Input('field-dropdown', 'value')])
def simple_graph(user_selection):
    df = get_data_object(user_selection)
    
    trace1 = go.Scatter(
        x = df['Actual'][:10],
        y = df.index[:10],
        name = 'Actual',
        mode = 'markers',
        marker = dict(size = 10,color = '#211a33')
    )
    
    trace2 = go.Scatter(
        x = df['Predictions'][:10],
        y = df.index[:10],
        name = 'Predictions',
        mode = 'markers',
        marker = dict(size = 10,color = '#e57200')
     )
    
    return {
        'data': [trace1, trace2],
        'layout': {'title': 'Simple Graph',
                   'xaxis': {'showticklabels': False, 'zeroline': False, 'showgrid': False},
                   'yaxis': {'showticklabels': False, 'zeroline': False, 'showgrid': False}
                  }
    }

@app.callback(Output('actual_histogram', 'figure'), [Input('field-dropdown', 'value')])
def update_actual_histogram(user_selection):
    df = get_data_object(user_selection)
    
    trace = go.Histogram(
        x = df['Actual Encoded'],
        xbins=dict(start=0, size=0.75, end=100),
                   marker=dict(color='#211a33'))
    
    return {
        'data': [trace],
        'layout': {'title': 'Actual Frequency Counts',
                   'xaxis': {'showticklabels': False},
                   'yaxis': {'showticklabels': False}
                  }
    }

@app.callback(Output('predictions_histogram', 'figure'), [Input('field-dropdown', 'value')])
def update_predictions_histogram(user_selection):
    df = get_data_object(user_selection)
    
    trace = go.Histogram(
        x = df['Predictions Encoded'],
        xbins=dict(start=0, size=0.75, end=100),
                   marker=dict(color='#e57200'))
    
    return {
        'data': [trace],
        'layout': {'title': 'Predictions Frequency Counts',
                   'xaxis': {'showticklabels': False},
                   'yaxis': {'showticklabels': False}
                  }
    }

@app.callback(Output('overlay_histogram', 'figure'), [Input('field-dropdown', 'value')])
def update_overlay_histogram(user_selection):
    df = get_data_object(user_selection)
    
    trace1 = go.Histogram(
        x = df['Actual Encoded'],
        xbins=dict(start=0, size=0.75, end=100),
                   marker=dict(color='#211a33'),
        opacity=0.75,
        name='Actual'
    )
    
    trace2 = go.Histogram(
        x = df['Predictions Encoded'],
        xbins=dict(start=0, size=0.75, end=100),
                   marker=dict(color='#e57200'),
        opacity=0.75,
        name='Predictions'
    )
    
    return {
        'data': [trace1, trace2],
        'layout': {'title': 'Overlay Frequency Counts',
                   'xaxis': {'showticklabels': False},
                   'yaxis': {'showticklabels': False}
                  }
    }

@app.callback(Output('density_graph', 'figure'), [Input('field-dropdown', 'value')])
def update_density_graph(user_selection):
    df = get_data_object(user_selection)

    trace = go.Scatter(
        x = df['Actual Encoded'],
        mode='markers',
        marker=dict(
            color=df['Predictions Encoded'],
            colorscale='Viridis',
            showscale=True
        )
    )
    
    return {
        'data': [trace],
        'layout': {'title': 'Density Graph',
                   'xaxis': {'showticklabels': True},
                   'yaxis': {'showticklabels': True}
                  }
    }

#### Run Application

In [ ]:
external_css = [
    "https://cdnjs.cloudflare.com/ajax/libs/normalize/7.0.0/normalize.min.css",
    "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
    "https://fonts.googleapis.com/css?family=Open+Sans|Roboto",
    "https://codepen.io/mrushin/pen/bQVEgg.css",
]

for css in external_css:
    app.css.append_css({"external_url": css})

if __name__ == '__main__':
    app.run_server(debug=False)